In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import pickle

app = Flask(__name__)

# Load the trained model
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

# Create an empty DataFrame to store the data
data_df = pd.DataFrame(columns=['customerOrderID', 'salesOrg', 'distributionChannel', 'customerNumber',
                                'companyCode', 'orderCurrency'])

@app.route('/predict', methods=['POST'])
def predict():
    # Get the input data from the request
    data = request.get_json()

    # Extract the input features
    customer_order_id = data['customerOrderID']
    sales_org = data['salesOrg']
    distribution_channel = data['distributionChannel']
    customer_number = data['customerNumber']
    company_code = data['companyCode']
    order_currency = data['orderCurrency']

    # Add the data to the DataFrame
    data_df.loc[len(data_df)] = [customer_order_id, sales_org, distribution_channel, customer_number,
                                 company_code, order_currency]

    # Perform any necessary preprocessing on the input data
    # ...

    # Create a feature vector using the preprocessed data
    features = [[customer_order_id, sales_org, distribution_channel, customer_number, company_code, order_currency]]

    # Make a prediction using the loaded model
    prediction = model.predict(features)

    # Format the prediction as a response
    response = {'amountInUSD': float(prediction)}

    return jsonify(response)

if __name__ == '__main__':
    app.run()


In [ ]:
from scipy.stats import skew
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np

# Calculate skewness before log transformation
skewness_before = skew(data_df['RELEASED_CREDIT_VALUE'])

# Apply log transformation
data_df['RELEASED_CREDIT_VALUE'] = np.log(data_df['RELEASED_CREDIT_VALUE'])

# Calculate skewness after log transformation
skewness_after = skew(data_df['RELEASED_CREDIT_VALUE'])

# Print the skewness values
print("Skewness before log transformation:", skewness_before)
print("Skewness after log transformation:", skewness_after)


label_encoder = LabelEncoder()

# Convert 'PURCHASE_ORDER_TYPE' column to string
data_df['PURCHASE_ORDER_TYPE'] = data_df['PURCHASE_ORDER_TYPE'].astype(str)

# Encode 'PURCHASE_ORDER_TYPE'
data_df['PURCHASE_ORDER_TYPE'] = label_encoder.fit_transform(data_df['PURCHASE_ORDER_TYPE'])

# Encode 'CREDIT_CONTROL_AREA'
data_df['CREDIT_CONTROL_AREA'] = label_encoder.fit_transform(data_df['CREDIT_CONTROL_AREA'])

# Encode 'SELLING_BUSINESS_SUBBRANCH'
data_df['SELLING_BUSINESS_SUBBRANCH'] = label_encoder.fit_transform(data_df['SELLING_BUSINESS_SUBBRANCH'])

# Encode 'DELIVERY_ADDRESS'
data_df['DELIVERY_ADDRESS'] = label_encoder.fit_transform(data_df['DELIVERY_ADDRESS'])


median_value = data_df['amount_in_usd'].median()
data_df['amount_in_usd'] = data_df['amount_in_usd'].fillna(median_value)
data_df['amount_in_usd'] = data_df['amount_in_usd'].replace([np.inf, -np.inf], median_value)


print(type(data_df['amount_in_usd']))  # Print the type of 'amount_in_usd'

data_df['amount_in_usd'] = data_df['amount_in_usd'].astype(int)  # Convert 'amount_in_usd' to integer


# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Normalize the 'amount_in_usd' column
data_df['amount_in_usd'] = scaler.fit_transform(data_df[['amount_in_usd']])


# Replace infinite values with NaN
data_df['amount_in_usd'] = data_df['amount_in_usd'].replace([np.inf, -np.inf], np.nan)

# Calculate the median excluding NaN values
median_value = data_df['amount_in_usd'].median(skipna=True)

# Replace NaN values with the median
data_df['amount_in_usd'] = data_df['amount_in_usd'].fillna(median_value)


# Calculate quartiles
q1 = data_df['ORDER_AMOUNT'].quantile(0.25)
q3 = data_df['ORDER_AMOUNT'].quantile(0.75)

# Calculate interquartile range (IQR)
iqr = q3 - q1

# Calculate lower and upper whiskers
lower_whisker = q1 - 1.5 * iqr
upper_whisker = q3 + 1.5 * iqr

# Replace outliers with imputed values
data_df.loc[data_df['ORDER_AMOUNT'] < lower_whisker, 'ORDER_AMOUNT'] = data_df['ORDER_AMOUNT'].median()
data_df.loc[data_df['ORDER_AMOUNT'] > upper_whisker, 'ORDER_AMOUNT'] = data_df['ORDER_AMOUNT'].median()


# Replace infinite values with NaN
data_df['RELEASED_CREDIT_VALUE_LOG'] = data_df['RELEASED_CREDIT_VALUE_LOG'].replace([np.inf, -np.inf], np.nan)
data_df['ORDER_AMOUNT'] = data_df['ORDER_AMOUNT'].replace([np.inf, -np.inf], np.nan)
data_df['CREDIT_STATUS'] = data_df['CREDIT_STATUS'].replace([np.inf, -np.inf], np.nan)

# Calculate the median excluding NaN values
released_credit_median = data_df['RELEASED_CREDIT_VALUE_LOG'].median(skipna=True)
order_amount_median = data_df['ORDER_AMOUNT'].median(skipna=True)
credit_status_median = data_df['CREDIT_STATUS'].median(skipna=True)

# Replace NaN values with the median
data_df['RELEASED_CREDIT_VALUE_LOG'] = data_df['RELEASED_CREDIT_VALUE_LOG'].fillna(released_credit_median)
data_df['ORDER_AMOUNT'] = data_df['ORDER_AMOUNT'].fillna(order_amount_median)
data_df['CREDIT_STATUS'] = data_df['CREDIT_STATUS'].fillna(credit_status_median)


In [ ]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# Load the trained model
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

@app.route('/predict', methods=['POST'])
def predict():
    # Get the input data from the request
    data = request.get_json()

    # Extract the input features
    customer_order_id = data['customerOrderID']
    sales_org = data['salesOrg']
    distribution_channel = data['distributionChannel']
    customer_number = data['customerNumber']
    company_code = data['companyCode']
    order_currency = data['orderCurrency']

    # Perform any necessary preprocessing on the input data
    # ...

    # Create a feature vector using the preprocessed data
    features = [[customer_order_id, sales_org, distribution_channel, customer_number, company_code, order_currency]]

    # Make a prediction using the loaded model
    prediction = model.predict(features)

    # Format the prediction as a response
    response = {'amountInUSD': float(prediction)}

    return jsonify(response)

if __name__ == '__main__':
    app.run()
